### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
성능 개선을 위한 테스트에서는 전체를 대상으로 인덱싱하지 않고, 5개 인덱싱 코드 실행 권장(주석 해제)

In [1]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

/home/ohs3201/codeit/codeit-part3-team4/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
BASE_DIR = Path.cwd().parent  # /codeit-part3-team4
RAW_FOLDER = BASE_DIR / "data/raw/files"  # 수정된 경로

# PDF to text
def extract_text(pdf_path: Path | str) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    return "\n".join(texts)

# 폴더에서 PDF 목록 가져오기
def get_pdf_paths(folder_path: Path | str) -> list[Path]:
    folder = Path(folder_path)
    pdf_paths = [p for p in folder.glob("*.pdf")]
    return sorted(pdf_paths)

In [4]:
# Chunking
def chunk(text: str, size: int = 800) -> list[str]:
    return [text[i:i+size] for i in range(0, len(text), size)]

In [5]:
# 임베딩 및 인덱스 만들기
# 한국어 임베딩 모델
embed_model_name = "nlpai-lab/KoE5"
embed_model = SentenceTransformer(embed_model_name)

def build_index(chunks: list[str]):
    embs = embed_model.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
    index = faiss.IndexFlatL2(embs.shape[1])
    index.add(embs.astype("float32"))
    return index, chunks

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 558.72it/s, Materializing param=pooler.dense.weight]                               


In [6]:
# 질문 리스트 (key, 질문)
queries = [
    ("project_name", "사업(용역)명은 무엇인가?"),
    ("agency", "발주 기관(수요기관)은 어디인가?"),
    ("purpose", "사업 목적(추진 배경)은 무엇인가?"),
    ("budget", "총 사업 예산(사업비)은 얼마인가?"),
    ("contract_type", "계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?"),
    ("deadline", "입찰/제안서 제출 마감일시는 언제인가?"),
    ("duration", "사업 수행 기간은 얼마나 되는가?"),
    ("requirements_must", "필수 요구사항(기능/성능/보안 등)은 무엇인가?"),
    ("eval_items", "평가 항목(기술/가격 등) 구성은 어떻게 되는가?"),
    ("price_eval", "가격 평가 방식(최저가/협상 등)은 무엇인가?"),
    ("eligibility", "참가 자격 요건(면허/실적/인증/등급)은 무엇인가?"),
]

def build_query_prompt(queries):
    return "\n".join(
        f"{i}. {q}"
        for i, (_, q) in enumerate(queries, start=1)
    )

In [7]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

[분석 규칙]
- 추측 금지, 문서에 명시된 내용만 사용
- 문서에 없으면 반드시 "NOT_FOUND"
- **출력은 질문 개수와 동일한 줄 수**
- **각 줄에는 답변 텍스트만 작성**
- 번호, 질문 문장, '답:', 기호, 설명을 절대 포함하지 말 것

[질문 목록]
{questions}

[컨텍스트]
{context}

질문 순서대로 답변만 한 줄씩 출력하라.
"""

### 배치 질문 응답

질문 리스트를 한 번에 임베딩/검색하고, top-k 청크를 합쳐 LLM에 넣어 답변을 생성합니다.
`return_indices=True`일 때는 검색된 청크 인덱스도 반환합니다.

In [8]:
def answer_batch(index, model, chunks, queries, top_k: int = 15, return_indices: bool = False) -> list[str]:
    q_list = [q for _, q in queries]
    q_emb = model.encode(q_list)
    q_mean = q_emb.mean(axis=0, keepdims=True)
    _, I = index.search(q_mean.astype("float32"), top_k)
    context = "".join(chunks[i] for i in I[0])

    prompt = RFP_PROMPT.format(questions=build_query_prompt(queries), context=context)
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )
    text = (resp.choices[0].message.content or "").strip()
    answers = [line.strip() for line in text.splitlines() if line.strip()]
    if return_indices:
        return answers, [int(i) for i in I[0]]
    return answers

In [9]:
# 쿼리 답변
def answer(index, embed_model, chunks, queries, top_k: int = 15) -> dict:
    # 1. 검색용 대표 질문
    search_query = "RFP 주요 사업 정보 요약"
    q_emb = embed_model.encode([search_query], convert_to_numpy=True)
    _, I = index.search(q_emb.astype("float32"), top_k)
    context = "\n\n".join(chunks[i] for i in I[0])
    questions_text = build_query_prompt(queries)
    prompt = RFP_PROMPT.format(
        context=context,
        questions=questions_text
    )
    # 2. OpenAI API 호출
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
        temperature=0.0,
    )
    raw_answer = resp.choices[0].message.content.strip()
    # 3. key: value 매핑
    results = {}
    for (key, _), line in zip(queries, raw_answer.splitlines()):
        line = line.strip()
        if line:
            results[key] = line
        else:
            results[key] = "명시 없음"
    return results

In [10]:
# 데이터 폴더 설정
docs = get_pdf_paths(RAW_FOLDER)
print(f"발견된 PDF: {len(docs)}개")
for i, doc in enumerate(docs):
    print(f"{i}: {doc.name}")

발견된 PDF: 100개
0: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
1: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
2: (사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
3: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
4: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
6: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
7: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
8: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
9: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
10: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
11: 경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf
12: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
13: 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.pdf
14: 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
15: 광주과학기술원_학사시스템 기능개선 사업.pdf
16: 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
17: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
18: 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
19: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
20: 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
21: 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.pdf
22: 국방과학연구소_기록관리시스

In [11]:
# 전체 문서별 인덱스 저장
doc_indexes = {}
for doc_path in docs:
    print(f"처리 중: {doc_path.name}")
    text = extract_text(doc_path)
    chunks = chunk(text)
    index, chunks_list = build_index(chunks)
    doc_indexes[doc_path] = (index, chunks_list)
print("모든 문서 인덱싱 완료")

처리 중: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
처리 중: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
처리 중: (사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
처리 중: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
처리 중: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
처리 중: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
처리 중: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
처리 중: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
처리 중: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
처리 중: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
처리 중: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
처리 중: 경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf
처리 중: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
처리 중: 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.pdf
처리 중: 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
처리 중: 광주과학기술원_학사시스템 기능개선 사업.pdf
처리 중: 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
처리 중: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
처리 중: 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
처리 중: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
처리 중: 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
처리 중: 국민연금공단_사업장 사회보험료 지원 

In [12]:
# 전체 중 5개 문서만 문서별 인덱스 저장
doc_indexes = {}
for doc_path in docs[:5]:
    print(f"처리 중: {doc_path.name}")
    text = extract_text(doc_path)
    chunks = chunk(text)
    index, chunks_list = build_index(chunks)
    doc_indexes[doc_path] = (index, chunks_list)
print("5개 문서 인덱싱 완료")

처리 중: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
처리 중: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
처리 중: (사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
처리 중: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
처리 중: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5개 문서 인덱싱 완료


In [13]:
# 테스트: 단일 문서
if docs:
    test_doc = docs[0]
    print(f"\n=== {test_doc.name} 분석 ===")
    index, chunks = doc_indexes[test_doc]
    result = answer(index, embed_model, chunks, queries)
    for k, v in result.items():
        print(f"{k}: {v}")


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: NOT_FOUND
agency: NOT_FOUND
purpose: 복수의결권주식, 스톡옵션 시스템 고도화 및 신규 구축업무 추진
budget: 20억원 미만
contract_type: 협상에 의한 계약
deadline: 2024년 3월
duration: NOT_FOUND
requirements_must: NOT_FOUND
eval_items: 개발장비, 통제 및 리스크 계획
price_eval: 기술평가(90%), 가격평가(10%)
eligibility: 최저가


In [14]:
# 테스트: 전체 문서(비용 주의-주석 해제 후 사용)

import json

results = {}
for doc in docs:
    print(f"\n=== {doc.name} 분석 ===")
    if doc not in doc_indexes:
        print(f"[SKIP] 인덱스 없음: {doc.name}")
        continue
    index, chunks = doc_indexes[doc]
    result = answer(
        index=index,
        embed_model=embed_model,
        chunks=chunks,
        queries=queries
    )
    results[doc.name] = result
    for k, v in result.items():
        print(f"{k}: {v}")

# JSON 저장
output_path = BASE_DIR / "rfp_api_answer_results_seok.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f"\n전체 문서 분석 완료! 저장 위치: {output_path}")


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: NOT_FOUND
agency: NOT_FOUND
purpose: 복수의결권 및 스톡옵션 시스템 고도화
budget: 20억원 미만
contract_type: 협상에 의한 계약
deadline: 2024년 3월
duration: NOT_FOUND
requirements_must: NOT_FOUND
eval_items: 개발장비, 통제 및 리스크 계획
price_eval: 기술평가(90%), 가격평가(10%)
eligibility: 최저가

=== (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf 분석 ===
project_name: BIFF&ACFM 온라인서비스 재개발 및 행사지원시스템 공급 사업
agency: 부산국제영화제
purpose: 웹 표준, 웹 접근성 및 웹 UI에 대한 기능개선으로 사용자 중심의 웹서비스를 제공한다.
budget: 금243,000,000원(금이억사천삼백만 원) (VAT 포함)
contract_type: 협상에 의한 계약
deadline: 2024. 7. 2(화) 16:00까지
duration: 계약체결일 ~ 2025. 05. 31.
requirements_must: NOT_FOUND
eval_items: NOT_FOUND
price_eval: NOT_FOUND
eligibility: NOT_FOUND

=== (사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf 분석 ===
project_name: KUSF 체육특기자 경기기록 관리시스템 개발
agency: 한국대학스포츠협의회
purpose: 현행 대입 체육특기자 선발에 경기실적으로만 당락을 결정하는 문제 해결을 위한 가이드라인 및 권고 발표
budget: 1억 5천만 원(부가가치세 포함)
contract_type: 협상에 의한 계약
deadline: 계약체결 후 2024년 12월 31일까지


## RAG 평가 (docs_api_summary 베이스라인 그대로 + gold로 점수 측정)

In [15]:
import json
import pandas as pd
from tqdm.auto import tqdm
from rapidfuzz import fuzz

In [16]:
# 평가 데이터 경로
RAW_FILES_DIR = BASE_DIR / "data/raw/files"
EVAL_DIR = BASE_DIR / "data/raw/eval"

GOLD_EVIDENCE_CSV = EVAL_DIR / "gold_evidence.csv"
GOLD_FIELDS_JSONL = EVAL_DIR / "gold_fields.jsonl"

print("BASE_DIR:", BASE_DIR)
print("RAW_FILES_DIR:", RAW_FILES_DIR, "exists:", RAW_FILES_DIR.exists())
print("EVAL_DIR:", EVAL_DIR, "exists:", EVAL_DIR.exists())

BASE_DIR: /home/ohs3201/codeit/codeit-part3-team4
RAW_FILES_DIR: /home/ohs3201/codeit/codeit-part3-team4/data/raw/files exists: True
EVAL_DIR: /home/ohs3201/codeit/codeit-part3-team4/data/raw/eval exists: True


### 평가용 정답 데이터 불러오기

In [19]:
def load_gold_fields_jsonl(path: Path) -> pd.DataFrame:
    rows = []
    with path.open("r", encoding="utf-8", errors="replace") as f:
        for line in f:
            if not line.strip():
                continue
            rows.append(json.loads(line))
    out = []
    for r in rows:
        iid = r["instance_id"]
        doc_id = r.get("doc_id", "")
        fields = r.get("fields", {}) or {}
        for k, v in fields.items():
            out.append({"instance_id": iid, "doc_id": doc_id, "field": k, "gold": v})
    return pd.DataFrame(out)


gold_evidence_df = pd.read_csv(GOLD_EVIDENCE_CSV)
gold_fields_df = load_gold_fields_jsonl(GOLD_FIELDS_JSONL)

display(gold_evidence_df.head(3))
display(gold_fields_df.head(3))

,instance_id,doc_id,page_start,page_end,anchor_text
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,4,4,사업개요
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,1,1,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,3,3,추진배경 및 방향


,instance_id,doc_id,field,gold
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,project_name,벤처확인종합관리시스템 기능 고도화
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,agency,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,purpose,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능..."


### Retrieval 평가(Anchor 기반)

`gold_evidence.csv`의 `anchor_text`가 top-k 청크 안에 포함되는지로 retrieval을 평가합니다.
- `recall@k`: top-k 안에서 anchor가 한 번이라도 나오면 1
- `mrr@k`: 첫 hit의 역순위(없으면 0)

In [20]:
def build_gold_anchor_map(df: pd.DataFrame) -> dict[str, list[str]]:
    m: dict[str, list[str]] = {}
    for _, r in df.iterrows():
        iid = str(r["instance_id"])
        anchor = str(r.get("anchor_text", "") or "").strip()
        if anchor:
            m.setdefault(iid, []).append(anchor)
    return m

GOLD_ANCHOR = build_gold_anchor_map(gold_evidence_df)


def eval_retrieval_by_anchor(chunks: list[str], idxs: list[int], anchors: list[str]) -> dict[str, float]:
    hit_rank = None
    for rank, ci in enumerate(idxs, start=1):
        if 0 <= int(ci) < len(chunks):
            c = chunks[int(ci)]
            if any(a in c for a in anchors):
                hit_rank = rank
                break
    return {"recall": 1.0 if hit_rank else 0.0, "mrr": (1.0 / hit_rank) if hit_rank else 0.0}

In [21]:
# 베이스라인 질문 리스트 (docs_api_summary의 queries 사용)
# queries 변수는 기존 노트북 셀에 정의되어 있어야 합니다.
BASELINE_QUERIES = queries
BASELINE_QUESTIONS = dict(BASELINE_QUERIES)

### 정답 유사도 평가 함수

In [22]:
def eval_gen(pred: str, gold: str | None, threshold: int = 80) -> dict[str, float]:
    pred = (pred or "").strip()
    fill = 1.0 if pred and pred.lower() not in {"", "없음"} else 0.0
    if gold is None or str(gold).strip() == "":
        return {"fill": fill, "match": np.nan, "sim": np.nan}
    gold = str(gold).strip()
    sim = fuzz.token_set_ratio(pred, gold)
    return {"fill": fill, "match": 1.0 if sim >= threshold else 0.0, "sim": float(sim)}

In [23]:
SIM_THRESHOLD = 80
TOP_K = 15

In [24]:
# docs_api_summary에서 만든 docs 리스트 사용
# docs = get_pdf_paths(RAW_FOLDER) 가 이미 실행되어 있어야 합니다.
DOC_PATHS = docs
for p in DOC_PATHS:
    print(" -", p.name)

 - (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
 - (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
 - (사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
 - (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
 - 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
 - BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
 - KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
 - 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
 - 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
 - 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
 - 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
 - 경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf
 - 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
 - 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.pdf
 - 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
 - 광주과학기술원_학사시스템 기능개선 사업.pdf
 - 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
 - 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
 - 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
 - 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
 - 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
 - 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.pdf
 - 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.pdf
 - 국

### 문서 단위 평가

- generation: `gen_fill_rate`, `gen_match_rate`, `gen_avg_similarity`
- retrieval: `retrieval_recall@k`, `retrieval_mrr@k` (anchor 기반)
- score: 위 지표의 가중합

In [25]:
def run_eval_for_doc(doc_path: Path) -> pd.DataFrame:
    doc_name = doc_path.name
    qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()
    if len(qdf) == 0:
        print("⚠️ no gold fields:", doc_name)
        return pd.DataFrame()

    # 베이스라인 청킹/인덱싱
    text = extract_text(doc_path)
    chunks = chunk(text)
    index, chunks = build_index(chunks)
    model = embed_model

    # 배치 답변 생성 (베이스라인 질문 순서)
    answers, idxs = answer_batch(index, model, chunks, BASELINE_QUERIES, top_k=TOP_K, return_indices=True)
    answer_map = {k: answers[i] if i < len(answers) else "" for i, (k, _) in enumerate(BASELINE_QUERIES)}

    # 평가
    r_list, g_list = [], []
    for _, row in qdf.iterrows():
        iid = str(row["instance_id"])
        field = str(row["field"])
        gold = row["gold"]
        pred = answer_map.get(field, "")

        anchors = GOLD_ANCHOR.get(iid, [])
        if anchors:
            r_list.append(eval_retrieval_by_anchor(chunks, idxs, anchors))
        else:
            r_list.append({"recall": np.nan, "mrr": np.nan})

        g_list.append(eval_gen(pred, gold, threshold=SIM_THRESHOLD))

    rows = [{
        "doc_id": doc_name,
        "n_questions_total": len(qdf),
        "n_questions_with_evidence": int(sum(1 for x in qdf["instance_id"] if str(x) in GOLD_ANCHOR)),
        "retrieval_recall@k": float(np.nanmean([x["recall"] for x in r_list])),
        "retrieval_mrr@k": float(np.nanmean([x["mrr"] for x in r_list])),
        "n_questions_with_gold_fields": len(qdf),
        "gen_fill_rate": float(np.nanmean([x["fill"] for x in g_list])),
        "gen_match_rate": float(np.nanmean([x["match"] for x in g_list])),
        "gen_avg_similarity": float(np.nanmean([x["sim"] for x in g_list])),
    }]
    return pd.DataFrame(rows)


all_results = []
for doc_path in DOC_PATHS:
    all_results.append(run_eval_for_doc(doc_path))

results_df = pd.concat([d for d in all_results if len(d)], ignore_index=True) if all_results else pd.DataFrame()

if len(results_df):
    results_df["score"] = (
        0.5 * results_df["retrieval_mrr@k"].fillna(0) +
        0.2 * results_df["retrieval_recall@k"].fillna(0) +
        0.3 * results_df["gen_match_rate"].fillna(0)
    )

display(results_df)

if len(results_df):
    out_path = BASE_DIR / "outputs" / "eval_grid_results_docs_base_ver2.csv"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    results_df.to_csv(out_path, index=False, encoding="utf-8-sig")
    display(results_df.sort_values(["retrieval_mrr@k", "gen_match_rate"], ascending=[False, False]).head(20))
    print("Saved:", out_path)

⚠️ no gold fields: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
⚠️ no gold fields: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
⚠️ no gold fields: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
⚠️ no gold fields: 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.pdf
⚠️ no gold fields: 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
⚠️ no gold fields: 광주과학기술원_학사시스템 기능개선 사업.pdf
⚠️ no gold fields: 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
⚠️ no gold fields: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
⚠️ no gold fields: 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
⚠️ no gold fields: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
⚠️ no gold fields: 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
⚠️ no gold fields: 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.pdf
⚠️ no gold fields: 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.pdf
⚠️ no gold fields: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
⚠️ no gold fields: 부산관광공사_경영정보시스템 기능개선.pdf
⚠️ no gold fields: 사단법인 보험개발원_실손보험 청구 전산화 시스템 구축 사업.pdf
⚠️ no gold fields: 사단법인아시아물위원회사무국_우즈벡-키르기즈스탄 기후변화대응 스.pdf
⚠️ no gold fields: 서민금융진흥원_서민금

,doc_id,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,21,21,0.571429,0.189021,21,0.52381,0.095238,15.608625,0.237368
1,(사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...,21,21,1.000000,0.523810,21,0.52381,0.142857,23.826528,0.504762
2,(사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf,21,21,0.809524,0.559524,21,0.52381,0.238095,25.271240,0.513095
3,(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf,21,21,0.904762,0.469841,21,0.52381,0.190476,25.032073,0.473016
4,2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf,21,21,0.380952,0.234580,21,0.52381,0.095238,15.279445,0.222052
5,경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf,21,21,0.714286,0.382900,21,0.52381,0.095238,23.503899,0.362879
6,경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf,21,21,0.571429,0.129649,21,0.52381,0.190476,23.714445,0.236253
7,경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf,21,21,0.571429,0.197802,21,0.47619,0.142857,21.821060,0.256044
8,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf,21,21,0.571429,0.301701,21,0.52381,0.095238,22.029898,0.293707
9,경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf,21,21,0.761905,0.413492,21,0.52381,0.238095,31.952207,0.430556


,doc_id,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
16,대한상공회의소_기업 재생에너지 지원센터 홈페이지 개편 및 시스템 고.pdf,21,21,0.714286,0.591631,21,0.52381,0.285714,35.815774,0.524387
2,(사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf,21,21,0.809524,0.559524,21,0.52381,0.238095,25.271240,0.513095
10,국방과학연구소_대용량 자료전송시스템 고도화.pdf,21,21,0.761905,0.525397,21,0.52381,0.238095,32.368751,0.486508
1,(사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...,21,21,1.000000,0.523810,21,0.52381,0.142857,23.826528,0.504762
11,그랜드코리아레저(주)_2024년도 GKL 그룹웨어 시스템 구축 용역.pdf,21,21,0.714286,0.501684,21,0.52381,0.190476,26.762231,0.450842
3,(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf,21,21,0.904762,0.469841,21,0.52381,0.190476,25.032073,0.473016
9,경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf,21,21,0.761905,0.413492,21,0.52381,0.238095,31.952207,0.430556
5,경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf,21,21,0.714286,0.382900,21,0.52381,0.095238,23.503899,0.362879
15,대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.pdf,21,21,0.666667,0.367684,21,0.52381,0.142857,28.003815,0.360033
19,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,21,21,0.619048,0.349567,21,0.52381,0.285714,35.475424,0.384307


Saved: /home/ohs3201/codeit/codeit-part3-team4/outputs/eval_grid_results_docs_base_ver2.csv


### 질문 샘플 및 답변

In [26]:
# 질문 샘플 + 모델 답변 보기 (문서 1개 기준)
doc_name = results_df.sort_values("gen_match_rate").iloc[0]["doc_id"]  # 예시: 성능 낮은 문서
sample_n = 5

# 문서 선택
doc_path = next(p for p in DOC_PATHS if p.name == doc_name)

# 준비
qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()
text = extract_text(doc_path)
chunks = chunk(text)
index, chunks = build_index(chunks)
model = embed_model

# 배치 답변
answers, idxs = answer_batch(index, model, chunks, BASELINE_QUERIES, top_k=TOP_K, return_indices=True)
answer_map = {k: answers[i] if i < len(answers) else "" for i, (k, _) in enumerate(BASELINE_QUERIES)}

# 샘플 출력
rows = []
for _, row in qdf.head(sample_n).iterrows():
    field = str(row["field"])
    gold = row["gold"]
    pred = answer_map.get(field, "")
    question = dict(BASELINE_QUERIES).get(field, "")
    rows.append({"field": field, "question": question, "gold": gold, "pred": pred})

display(pd.DataFrame(rows))

,field,question,gold,pred
0,project_name,사업(용역)명은 무엇인가?,아·태 사이버범죄 역량강화 허브(APC-HUB) 홈페이지 및 온라인교육시스템 고도화 2차,NOT_FOUND
1,agency,발주 기관(수요기관)은 어디인가?,대검찰청,NOT_FOUND
2,purpose,사업 목적(추진 배경)은 무엇인가?,"[장소·시간에 구애받지 않는 온라인교육시스템 고도화, 아·태 지역 협력의 중심점 정...",NOT_FOUND
3,budget,총 사업 예산(사업비)은 얼마인가?,"35,750천원(부가가치세 포함)",NOT_FOUND
4,contract_type,계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?,"제한경쟁입찰, 협상에 의한 계약","제한경쟁입찰, 협상에 의한 계약"


In [27]:
# 질문 샘플 + 모델 답변 보기 (문서 1개 기준)
doc_name = "(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf"
sample_n = 5

# 문서 선택
doc_path = next(p for p in DOC_PATHS if p.name == doc_name)

# 준비
qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()
text = extract_text(doc_path)
chunks = chunk(text)
index, chunks = build_index(chunks)
model = embed_model

# 배치 답변
answers, idxs = answer_batch(index, model, chunks, BASELINE_QUERIES, top_k=TOP_K, return_indices=True)
answer_map = {k: answers[i] if i < len(answers) else "" for i, (k, _) in enumerate(BASELINE_QUERIES)}

# 샘플 출력
rows = []
for _, row in qdf.head(sample_n).iterrows():
    field = str(row["field"])
    gold = row["gold"]
    pred = answer_map.get(field, "")
    question = dict(BASELINE_QUERIES).get(field, "")
    rows.append({"field": field, "question": question, "gold": gold, "pred": pred})

display(pd.DataFrame(rows))

,field,question,gold,pred
0,project_name,사업(용역)명은 무엇인가?,벤처확인종합관리시스템 기능 고도화,NOT_FOUND
1,agency,발주 기관(수요기관)은 어디인가?,벤처기업확인기관,발주 기관(수요기관)은 NOT_FOUND
2,purpose,사업 목적(추진 배경)은 무엇인가?,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능...","사업 목적(추진 배경)은 벤처기업법에 따른 복수의결권주식, 스톡옵션 등의 고도화 및..."
3,budget,총 사업 예산(사업비)은 얼마인가?,"352,000,000원",총 사업 예산(사업비)은 20억원 미만
4,contract_type,계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?,제한경쟁입찰,계약 방식은 협상에 의한 계약
